<a href="https://colab.research.google.com/github/00bfff-m/programming/blob/main/com_ppmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display
import pandas as pd # 데이터를 저장하고 처리하는 패키지
import math # log 써야 해서 필요함 

#1. bag of words 만들기
def bow(li):
    a = []
    for i in range(len(li)):
        a.extend(li[i])
    a = list(set(a))
    return a

#2. word co-occurance matrix 생성
def com(bow, str_list, window):
    matrix = []
    for i in range(len(bow)):
        matrix.append(list(map(int, [0 for j in range(len(bow))]))) # matrix 생성      
    for i in range(len(matrix)): 
        for j in range(len(matrix[i])): # matrix 범위 지정
            for k in range(len(str_list)):
                for l in range(len(str_list[k])): # str_list 범위 지정
                    if bow[j] == str_list[k][l]: # bow속 단어가 str_list어디에 있는 지 찾기
                        for x in range(l-window, l+window+1): # context window 내에 str_list에서 찾는 단어가 있는 지 찾기
                            if x < 0 or x == l or x >= len(str_list[k]): #범위를 벗어나면 넘기기
                                continue
                            elif str_list[k][x] == bow[i]:
                                matrix[i][j] = matrix[i][j] + 1
    return matrix



# 3. pointwise mutual information(점 상호 정보량) matrix 생성
# 상호 정보량: 확률 변수 A, B가 서로 독립적이면 0 양의 상관관계를 가지면 +, 음의 상관관계를 가지면 -
# word co-occurance matrix는 빈도 기준 -> 관계 기준으로 치환 
# 단어 A, B가 있을 때, A와 B가 동시에 나올 확률 P(A&B), A가 나올 확률 P(A), B가 나올 확률 P(B)
# PPMI(AB) = log2(P(A&B)/P(A)*P(B)) -> PPMI가 높을수록 두 단어 사이의 관계가 밀접함
# ex) 이연복, 짬뽕, 맵다의 관계: 맵다는 단어 빈도수가 높지만 짬뽕에만 쓰이는 건 아님 vs 백종원은 빈도수가 낮지만 짬뽕에만 나옴
# 관계기반은 빈도수에 기반을 두는 게 아닌 상호 관계를 기반으로 판단함
# -> 빈도수로 따지면 전자가 더 높지만 PPMI값은 후자가 높음
 
def ppmi(com):
    total = 0
    for i in range(len(com)):
        total = total + sum(com[i])
    matrix = []
    for i in range(len(com)):
        matrix.append(list(map(int, [0 for j in range(len(com))])))    
    for i in range(len(com)):
        for j in range(len(com[i])):
            matrix[i][j] = com[i][j]/total
    pmatrix = [] # 기존 matrix에 덮어 씌우면 오류가 생기므로 새 matrix 생성 
    for i in range(len(com)):
        pmatrix.append(list(map(int, [0 for j in range(len(com))])))
    for i in range(len(com)):
        for j in range(len(com[i])):
            p_ij = matrix[i][j]
            p_j = sum(matrix[i])
            p_i = 0
            for k in range(len(matrix)):
                p_i = p_i+matrix[k][j]
            try:
                pmatrix[i][j] = math.log2(p_ij / (p_i * p_j))
            except ValueError:
                pmatrix[i][j] = 0
    return pmatrix
str1 = 'I like deep learning'
str2 = 'I like NLP'
str3 = 'I enjoy flying' # https://rueki.tistory.com/33
str_list = [str1.split(), str2.split(), str3.split()]

com = com(bow(str_list), str_list, 4)

display(pd.DataFrame(com, index=bow(str_list), columns=bow(str_list)))

display(pd.DataFrame(ppmi(com), index=bow(str_list), columns=bow(str_list)))

,learning,NLP,I,like,deep,flying,enjoy
learning,0,0,1,1,1,0,0
NLP,0,0,1,1,0,0,0
I,1,1,0,2,1,1,1
like,1,1,2,0,1,0,0
deep,1,0,1,1,0,0,0
flying,0,0,1,0,0,0,1
enjoy,0,0,1,0,0,1,0


,learning,NLP,I,like,deep,flying,enjoy
learning,0.000000,0.000000,0.192645,0.678072,1.415037,0.000000,0.000000
NLP,0.000000,0.000000,0.777608,1.263034,0.000000,0.000000,0.000000
I,0.192645,0.777608,0.000000,0.455679,0.192645,0.777608,0.777608
like,0.678072,1.263034,0.455679,0.000000,0.678072,0.000000,0.000000
deep,1.415037,0.000000,0.192645,0.678072,0.000000,0.000000,0.000000
flying,0.000000,0.000000,0.777608,0.000000,0.000000,0.000000,2.584963
enjoy,0.000000,0.000000,0.777608,0.000000,0.000000,2.584963,0.000000
